In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")


In [2]:
from openai import OpenAI
client_Session = OpenAI(api_key=openai_api_key)
client_Session

In [3]:
!pip install gradio

In [4]:
def get_Ai_tutor_response_stream (user_question):
    system_prompt = "You are a Product Manager with 30 years of experience in the US as SaaS PM. You are very patient and understanding, explain the questions that a fresher in PM askes in the domain of Product management and Business Management. Also suggest some Youtube videos to watch on the concept asked, books to read or any resource that might be helpful"
    try:
        stream=client_Session.chat.completions.create(model = "gpt-4o-mini", 
                                                      messages = [{"role":"system", "content":system_prompt},
                                                                  {"role" : "user", "content" :user_question}], 
                                                      temperature = 0.7, stream = True)
        #the value of temperature is between 0 to 2

        full_response=""
        for chunck in stream:
            if chunck.choices[0].delta and chunck.choices[0].delta.content:
                text_chunck= chunck.choices[0].delta.content
                full_response+=text_chunck
                yield full_response 
            #the yeild operator acts as streaming function

    except Exception as e:
        print("sorry, some error occurred",e)
        yield f"there is some error:{e}"

In [5]:
explanantion_levels = {1: "Like I'm 5 years old",
                       2: "Like I'm 10 years old",
                       3: "Like I'm Undergraduation student",
                       4: "Like I'm Masters student",
                       5: "Like I have completed Phd",
                       6: "Like I have 15+ years of experience in the industry"
                      }

In [19]:
# Let's create a new function that streams the response
def stream_ai_tutor_response(user_question, explanation_level_value):

    level_description = explanantion_levels.get(explanation_level_value, "clearly and concisely")
    system_prompt = f"You are a Product Manager with 30 years of experience in the US as SaaS PM. You are very patient and understanding, explain the questions that a fresher in PM askes in the domain of Product management and Business Management. Also suggest some Youtube videos to watch on the concept asked, books to read or any resource that might be helpful. Explain the concept {level_description}"

    print(f"DEBUG: Using System Prompt: '{system_prompt}'")  # For checking

    try:
        # Note: stream = True is the key change here!
        stream = client_Session.chat.completions.create(
            model = "gpt-4o-mini",
            messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_question}],
            temperature = 0.7,
            stream = True,  # Enable streaming (magic happens here)
        )

        # Iterate through the response chunks
        full_response = ""  # Keep track of the full response if needed later

        # Loop through each chunk of the response as it arrives
        for chunk in stream:
            # Check if this chunk contains actual text content
            if chunk.choices[0].delta and chunk.choices[0].delta.content:
                # Extract the text from this chunk
                text_chunk = chunk.choices[0].delta.content
                # Add this chunk to our growing response
                full_response += text_chunk
                # 'yield' is special - it sends the current state of the response to Gradio
                # This makes the text appear to be typing in real-time
                
                yield full_response

    except Exception as e:
        print(f"An error occurred during streaming: {e}")
        yield f"Sorry, I encountered an error: {e}"



In [21]:
# Define a helper function to display markdown nicely
from IPython.display import display, Markdown
def print_markdown(text):
    """Displays text as Markdown in Jupyter."""
    display(Markdown(text))

In [ ]:
import gradio as gr
Product_Ai_tutor = gr.Interface(fn=stream_ai_tutor_response,
             inputs = [gr.Textbox(lines = 4, placeholder = "Ask your questions...", label = "Your Question"),
                      gr.Slider(
                          minimum = 1, 
                          maximum = 5, 
                          step = 1, 
                          value = 3, 
                          label = "Explanation complexity, 1 the most simplest and 5 the most complex"
                          ),
                          ],
             #outputs = gr.Textbox(lines = 20,label = "Your AI Product Tutor's answer"),
            outputs = gr.Markdown(label="Your AI Product Tutor's answer",container = True, height = 500),
             title = "🤖 AI Product Tutor!",
             description = "Struggling with Product Management concepts? Meet your Product AI Tutor — ask anything and get clear, step-by-step guidance with the most practical and detailed explanations",
             allow_flagging = "never") 

Product_Ai_tutor.launch(share=True)

C:\Users\mrudu\anaconda3\Lib\site-packages\gradio\interface.py:418: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://056e621860b72629ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
DEBUG: Using System Prompt: 'You are a Product Manager with 30 years of experience in the US as SaaS PM. You are very patient and understanding, explain the questions that a fresher in PM askes in the domain of Product management and Business Management. Also suggest some Youtube videos to watch on the concept asked, books to read or any resource that might be helpful. Explain the concept Like I'm Undergraduation student'
